Assignment Details


Note: if you having trouble displaying the maps try running jupyter nbextension enable --py gmaps in your environment and retry.


Create a heat map that displays the humidity for every city from the part I of the homework.

Narrow down the DataFrame to find your ideal weather condition. For example:


- A max temperature lower than 80 degrees but higher than 70.


- Wind speed less than 10 mph.


- Zero cloudiness.


Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.


Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.


Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.


Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.



In [1]:
# Dependencies
import requests
import json
import gmaps
import pandas as pd
import numpy as np

# Google developer API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

Humidity Heatmaps for all the cities 

(Dataframe from WeatherPy.ipynb Part I)

In [21]:
###Get dataframe to show City Name (unduplicated): weather1_data ###
#import dataframe weather1_data from 1st part of assignment (WeatherPy.ipynb)
%store -r weather1_data
#drop duplicate cities (same data)
weather1_data=weather1_data.drop_duplicates(subset=["City Name"])
weather1_data

,City Name,Latitude,Longitude,Temperature,Latitude Humidity,Cloudiness,Wind Speed
1,hithadhoo,-20.841587,85.603908,82.76,68.0,77.0,3.94
2,sao filipe,13.293488,-34.982080,75.88,60.0,0.0,6.08
4,bluff,-71.214122,172.363302,57.00,84.0,64.0,18.01
5,saint-philippe,-48.409808,62.306578,75.02,65.0,47.0,4.61
6,norman wells,68.737352,-120.152631,-13.00,77.0,90.0,3.51
...,...,...,...,...,...,...,...
486,gravdal,70.454049,13.348788,24.80,74.0,75.0,5.75
488,souillac,-51.565186,73.401185,82.40,74.0,40.0,6.91
495,ous,61.793925,61.297353,-17.07,88.0,100.0,6.85
496,qasigiannguit,66.282569,-48.212406,19.40,42.0,75.0,9.22


In [3]:
# Plot Heatmap
fig = gmaps.figure()
fig

# Store latitude and longitude in locations
locations = weather1_data[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
Humidity = weather1_data["Latitude Humidity"]

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, dissipating=False, max_intensity=100, point_radius=2)
heat_layer

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

Map of the first hotel found within each city (with ideal conditions for travel destination) located within 5000 meters of your coordinates.

In [4]:
###Cities selected based on ideal travel destination: ideal_df
#Narrow down ideal conditions of city 

#Ideal temperature Range (in Farenheit)
ideal_min_temp=70 
ideal_max_temp=80
#Ideal wind speed: speed less than... (in mph)
ideal_max_speed=10
#Ideal cloudiness: cloudiness less than... (%)
ideal_cloudiness=25

ideal_df=weather1_data[(weather1_data["Temperature"]>=ideal_min_temp) & (weather1_data["Temperature"]<=ideal_max_temp) & (weather1_data["Wind Speed"]<=ideal_max_speed) & (weather1_data["Cloudiness"]<=ideal_cloudiness)]
ideal_df

#Dataframe sorted ascending by least windy city.
idealbylowwind_df=ideal_df.sort_values(by="Wind Speed", ascending=True)
idealbylowwind_df

ideal_df.shape

(11, 7)

In [5]:
##Nearest hotel nearest specified co-ordinates
#Print: Hotel Name, Address, City Name, Country.

#Create new dataframe containing the City, Latitude, Longitude, Hotel Name,and Country.
nearby_hotel=ideal_df[["City Name","Latitude","Longitude"]]
nearby_hotel["Hotel Name"]=""
nearby_hotel["Hotel Lat"]=""
nearby_hotel["Hotel Lng"]=""
nearby_hotel["Hotel Address"]=""
nearby_hotel["Place ID"]=""
nearby_hotel["Country"]=""
nearby_hotel

# Assign variable values for parameters
# target_search="hotels"
radius=5000 #This is in metres as given in API documentation
place="lodging"

#Set parameters (unchanged for each row)
params = {
    # "keyword": target_search,
    "type": place,
    "radius": radius,
    "key": g_key
}

# Find a hotel near the specified city (co-ordinates available already)
for index, row in nearby_hotel.iterrows():
    #Use latitude and longitude from df
    lat= row["Latitude"]
    lng= row["Longitude"]
    #params dictionary to update each iteration
    #change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    #Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #make request and print url
    name_address = requests.get(base_url, params=params)

    #convert to json
    name_address = name_address.json()
    print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing, use a try-except to skip any that are missing a data point.
    try:
        nearby_hotel.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Name error")
        nearby_hotel.loc[index,"Hotel Name"]=np.nan

    try:
        nearby_hotel.loc[index, "Hotel Lat"]=name_address["results"][0]["geometry"]["location"]["lat"]
        nearby_hotel.loc[index, "Hotel Lng"]=name_address["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        print("Hotel Lat or Lng error")
        nearby_hotel.loc[index,"Hotel Lat"]=np.nan
        nearby_hotel.loc[index,"Hotel Lng"]=np.nan

    try:
        nearby_hotel.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
    except (KeyError, IndexError):
        print("Hotel Address error")
        nearby_hotel.loc[index,"Hotel Address"]=np.nan

    try:
        nearby_hotel.loc[index, "Place ID"] = name_address["results"][0]["place_id"]   
    except (KeyError, IndexError):
        print("Place ID error")
        nearby_hotel.loc[index,"Place ID"]=np.nan

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Name error
Hotel Lat or Lng error
Hotel Address error
Place ID error
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Name error
Hotel Lat or Lng error
Hotel Address error
Place ID error
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Name error
Hotel Lat or Lng error
Hotel Address error
Place ID error
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Name error
Hotel Lat or Lng error
Hotel Address error
Place ID error
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Name error
Hotel Lat or Lng error
Hotel Address error
Place ID error
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Name error
Hotel Lat or Lng error
Hotel Address error
Place ID error
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONA

In [6]:
nearby_hotel
#Still missing country

,City Name,Latitude,Longitude,Hotel Name,Hotel Lat,Hotel Lng,Hotel Address,Place ID,Country
2,sao filipe,13.293488,-34.982080,NaN,NaN,NaN,NaN,NaN,
14,puerto ayora,-11.683789,-104.103328,NaN,NaN,NaN,NaN,NaN,
28,salalah,17.472249,52.185599,NaN,NaN,NaN,NaN,NaN,
117,pisco,-24.358740,-96.527155,NaN,NaN,NaN,NaN,NaN,
135,hermanus,-81.063215,5.489997,NaN,NaN,NaN,NaN,NaN,
227,mapastepec,14.809307,-93.248545,NaN,NaN,NaN,NaN,NaN,
253,hadgaon,19.437249,77.936485,Pakiza Kirana store,19.4323,77.9603,"Bhokar - Kinwat Road, Rahim colony, Himayatnagar",ChIJvSZv_6_30TsR-9lKEOp8RUk,
345,barranca,-3.225088,-76.122612,NaN,NaN,NaN,NaN,NaN,
377,khetia,21.632076,74.539849,Jay shree Krishna,21.6216,74.5207,"Grampanchayat, Brahmanpuri",ChIJVVudBVZp3zsRZcAE_Ah2rqA,
420,bodden town,17.986858,-80.050378,NaN,NaN,NaN,NaN,NaN,


In [8]:
######In order to fill the Country of dataframe nearby_hotel, Googles Place API is used as shown in this cell. #####

## Using Google Places API. Documentation: https://developers.google.com/places/web-service/details

#Parameters required: key, place_id
params = {
    "key": g_key
}

# Specify parameter Place ID
for index, row in nearby_hotel.iterrows():

    params["place_id"]=row["Place ID"]

    base_url="https://maps.googleapis.com/maps/api/place/details/json?"

    #make request and print url
    data = requests.get(base_url, params=params)

    #convert to json
    data_json = data.json()
    #print(json.dumps(data_json, indent=4, sort_keys=True))

    try:
        #Look through lists with for loop and find the type where its dictionary contains the country
        for x in data_json["result"]["address_components"]:
            if "country" in x["types"]: #contains country in types
                country_name=x["short_name"] #only the country initial is required 
                nearby_hotel.loc[index, "Country"] = country_name
                print(country_name)
            
    except (KeyError, IndexError):
        print("Hotel country Error")
        nearby_hotel.loc[index,"Country"]=np.nan


Hotel country error
Hotel country error
Hotel country error
Hotel country error
Hotel country error
Hotel country error
IN
Hotel country error
IN
Hotel country error
Hotel country error


In [9]:
#View Dataframe of the desirable cities and its nearest hotel for a holiday: nearby_hotel
nearby_hotel

,City Name,Latitude,Longitude,Hotel Name,Hotel Lat,Hotel Lng,Hotel Address,Place ID,Country
2,sao filipe,13.293488,-34.982080,NaN,NaN,NaN,NaN,NaN,NaN
14,puerto ayora,-11.683789,-104.103328,NaN,NaN,NaN,NaN,NaN,NaN
28,salalah,17.472249,52.185599,NaN,NaN,NaN,NaN,NaN,NaN
117,pisco,-24.358740,-96.527155,NaN,NaN,NaN,NaN,NaN,NaN
135,hermanus,-81.063215,5.489997,NaN,NaN,NaN,NaN,NaN,NaN
227,mapastepec,14.809307,-93.248545,NaN,NaN,NaN,NaN,NaN,NaN
253,hadgaon,19.437249,77.936485,Pakiza Kirana store,19.4323,77.9603,"Bhokar - Kinwat Road, Rahim colony, Himayatnagar",ChIJvSZv_6_30TsR-9lKEOp8RUk,IN
345,barranca,-3.225088,-76.122612,NaN,NaN,NaN,NaN,NaN,NaN
377,khetia,21.632076,74.539849,Jay shree Krishna,21.6216,74.5207,"Grampanchayat, Brahmanpuri",ChIJVVudBVZp3zsRZcAE_Ah2rqA,IN
420,bodden town,17.986858,-80.050378,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#Drop NaN rows
nearby_hotel.dropna(how='any', inplace=True)
nearby_hotel

#Reset index
nearby_hotel = nearby_hotel.reset_index(drop=True)
nearby_hotel

,City Name,Latitude,Longitude,Hotel Name,Hotel Lat,Hotel Lng,Hotel Address,Place ID,Country
0,hadgaon,19.437249,77.936485,Pakiza Kirana store,19.4323,77.9603,"Bhokar - Kinwat Road, Rahim colony, Himayatnagar",ChIJvSZv_6_30TsR-9lKEOp8RUk,IN
1,khetia,21.632076,74.539849,Jay shree Krishna,21.6216,74.5207,"Grampanchayat, Brahmanpuri",ChIJVVudBVZp3zsRZcAE_Ah2rqA,IN


In [20]:
# Plot Heatmap
fig = gmaps.figure()
fig

# Store latitude and longitude list in variable locations
locations = weather1_data[["Latitude", "Longitude"]]

# Store humidity list in variable humidity
Humidity = weather1_data["Latitude Humidity"]

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, dissipating=False, max_intensity=100, point_radius=2)
heat_layer

# Add heat layer to figure
fig.add_layer(heat_layer)

#---------
#Create info box at marker
info_box_template = """
<dl>
<dt><b>Name</b></dt>{Hotel Name}</dd>
<dt><b>City</b></dt>{City Name}</dd>
<dt><b>Country</b></dt>{Country}</dd>
</dl>
"""

# Store the DataFrame Rows
hotel_info = [info_box_template.format(**row) for index, row in nearby_hotel.iterrows()]
locations = nearby_hotel[["Hotel Lat", "Hotel Lng"]]

#--------
#Assign to markers variable the dataframe row information
markers = gmaps.marker_layer(locations=locations,info_box_content=hotel_info)

#Add markers (hotel info) to figure
fig.add_layer(markers)

#Display figure
fig


Figure(layout=FigureLayout(height='420px'))